In [20]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle

In [6]:
import multiprocessing
import gensim
import nltk
from collections import defaultdict
from gensim.test.utils import datapath
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases,Phraser

In [24]:
import pandas as pd

data = pd.read_csv('preprocessed_data.csv',index_col=None,header=0)
data['lemmatized'][525] = ''
data = shuffle(data)
msk = np.random.rand(len(df)) < 0.8
df.reset_index(inplace=True,drop=True)
train = data[msk]
test = data[~msk]
len(train)

c:\users\limri\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


993

In [25]:
import os
class MySentences(object):
    """MySentences is a generator to produce a list of tokenized sentences 
    
    Takes a list of numpy arrays containing documents.
    
    Args:
        arrays: List of arrays, where each element in the array contains a document.
    """
    def __init__(self, *arrays):
        self.arrays = arrays
 
    def __iter__(self):
        for array in self.arrays:
            for document in array:
                for sent in nltk.sent_tokenize(document):
                    yield nltk.word_tokenize(sent)

def get_word2vec(sentences, location):
    """Returns trained word2vec
    
    Args:
        sentences: iterator for sentences
        
        location (str): Path to save/load word2vec
    """
    if os.path.exists(location):
        print('Found {}'.format(location))
        model = gensim.models.Word2Vec.load(location)
        return model
    
    print('{} not found. training model'.format(location))
    model = gensim.models.Word2Vec(sentences, size=100, window=5, min_count=5, workers=4)
    print('Model done training. Saving to disk')
    model.save(location)
    return model

In [27]:
w2vec = get_word2vec(
    MySentences(
        train['lemmatized'].values, 
        #df_test['Text'].values  Commented for Kaggle limits
    ),
    'w2vmodel'
)

w2vmodel not found. training model


W0718 00:22:10.187960  6240 base_any2vec.py:1386] under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay


Model done training. Saving to disk


In [28]:
class MyTokenizer:
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        transformed_X = []
        for document in X:
            tokenized_doc = []
            for sent in nltk.sent_tokenize(document):
                tokenized_doc += nltk.word_tokenize(sent)
            transformed_X.append(np.array(tokenized_doc))
        return np.array(transformed_X)
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(word2vec.wv.syn0[0])

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = MyTokenizer().fit_transform(X)
        
        return np.array([
            np.mean([self.word2vec.wv[w] for w in words if w in self.word2vec.wv]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

In [29]:
mean_embedding_vectorizer = MeanEmbeddingVectorizer(w2vec)
mean_embedded = mean_embedding_vectorizer.fit_transform(train['lemmatized'])

c:\users\limri\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).


In [30]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.optimizers import Adam

# Use the Keras tokenizer
num_words = 2000
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(train['lemmatized'].values)

In [31]:
X = tokenizer.texts_to_sequences(train['lemmatized'].values)
X = pad_sequences(X, maxlen=2000)

In [32]:
# Build out our simple LSTM
embed_dim = 128
lstm_out = 196

# Model saving callback
ckpt_callback = ModelCheckpoint('keras_model', 
                                 monitor='val_loss', 
                                 verbose=1, 
                                 save_best_only=True, 
                                 mode='auto')

model = Sequential()
model.add(Embedding(num_words, embed_dim, input_length = X.shape[1]))
model.add(LSTM(lstm_out, recurrent_dropout=0.2, dropout=0.2))
model.add(Dense(4,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['categorical_crossentropy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 2000, 128)         256000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 788       
Total params: 511,588
Trainable params: 511,588
Non-trainable params: 0
_________________________________________________________________
None


In [3]:
from sklearn.model_selection import train_test_split

In [33]:
Y = pd.get_dummies(train['labels']).values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42, stratify=Y)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(794, 2000) (794, 4)
(199, 2000) (199, 4)


In [34]:
batch_size = 32
model.fit(X_train, Y_train, epochs=8, batch_size=batch_size, validation_split=0.2, callbacks=[ckpt_callback])

W0718 00:22:39.407334  6240 deprecation.py:323] From c:\users\limri\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 635 samples, validate on 159 samples
Epoch 1/8
635/635 [==============================] - 109s 172ms/step - loss: 1.1625 - categorical_crossentropy: 1.1625 - val_loss: 1.0297 - val_categorical_crossentropy: 1.0297

Epoch 00001: val_loss improved from inf to 1.02971, saving model to keras_model
Epoch 2/8
635/635 [==============================] - 108s 171ms/step - loss: 1.0916 - categorical_crossentropy: 1.0916 - val_loss: 1.0108 - val_categorical_crossentropy: 1.0108

Epoch 00002: val_loss improved from 1.02971 to 1.01084, saving model to keras_model
Epoch 3/8
635/635 [==============================] - 109s 171ms/step - loss: 1.0014 - categorical_crossentropy: 1.0014 - val_loss: 1.0128 - val_categorical_crossentropy: 1.0128

Epoch 00003: val_loss did not improve from 1.01084
Epoch 4/8
635/635 [==============================] - 108s 169ms/step - loss: 0.8256 - categorical_crossentropy: 0.8256 - val_loss: 0.9944 - val_categorical_crossentropy: 0.9944

Epoch 00004: val_loss impro

In [35]:
model = load_model('keras_model')

In [42]:
probas = model.predict(X_test)

In [46]:
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix

pred_indices = np.argmax(probas, axis=1)
classes = np.array(range(1, 10))
preds = classes[pred_indices]
print('Log loss: {}'.format(log_loss(classes[np.argmax(Y_test, axis=1)], probas)))
print('Accuracy: {}'.format(accuracy_score(classes[np.argmax(Y_test, axis=1)], preds)))

Log loss: 1.0512710873168618
Accuracy: 0.5879396984924623


In [47]:
# https://www.kaggle.com/reiinakano/basic-nlp-bag-of-words-tf-idf-word2vec-lstm